# The folllowing notebook builds a summary of data for the 3/25 release using the experiment table from AWS combined with the associated cells from LIMS

In [1]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache
import visual_behavior.database as db
import pandas as pd
import numpy as np
import itertools

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## get the experiment table from S3

In [3]:
data_storage_directory = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/production_cache'
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

experiment_table = cache.get_ophys_experiment_table().reset_index()

In [4]:
len(experiment_table)

1165

## get all of the corresponding entries in the LIMS `cell_rois` table

In [5]:
oeids = experiment_table['ophys_experiment_id'].unique()
query = 'select * from cell_rois where ophys_experiment_id in {}'.format(tuple(oeids))
lims_rois = db.lims_query(query)

## get the number of valid ROIs as those with non-null `cell_specimen_id` and `valid_roi == True`

In [6]:
valid_rois = lims_rois[pd.notnull(lims_rois['cell_specimen_id'])].query('valid_roi').copy()
print('there are {} valid ROIS across {} experiments'.format(len(valid_rois), len(valid_rois['ophys_experiment_id'].unique())))

there are 92209 valid ROIS across 1165 experiments


## get the number of unique `cell_specimen_id` values

In [7]:
len(valid_rois['cell_specimen_id'].unique())

34681

## merge in the experiment table to get identifying info about the experiment for each valid ROI
check that the length is the same before and after the merge

In [8]:
print('len(valid_rois) before merge = {}'.format(len(valid_rois)))
valid_rois = valid_rois.merge(
    experiment_table,
    left_on = ['ophys_experiment_id'],
    right_on = ['ophys_experiment_id'],
)
print('len(valid_rois) after merge = {}'.format(len(valid_rois)))

len(valid_rois) before merge = 92209
len(valid_rois) after merge = 92209


## add a column with the transgenic line

In [9]:
def make_transgenic_line(row):
    return row['cre_line'] + ';' + row['reporter_line'].split('(')[0]
valid_rois['transgenic_line'] = valid_rois.apply(make_transgenic_line, axis=1)

## Now get counts for various attributes, grouped by project code and transgenic line

In [10]:
mouse_count = pd.DataFrame(
    valid_rois.groupby(['project_code', 'transgenic_line'])['mouse_id'].nunique()
).rename(columns = {'mouse_id': 'number of mice'})

In [11]:
session_count = pd.DataFrame(
    valid_rois.groupby(['project_code', 'transgenic_line'])['ophys_session_id'].nunique()
).rename(columns = {'ophys_session_id': 'number of sessions'})

In [12]:
experiment_count = pd.DataFrame(
    valid_rois.groupby(['project_code', 'transgenic_line'])['ophys_experiment_id'].nunique()
).rename(columns = {'ophys_experiment_id': 'number of experiments'})

In [13]:
roi_count = pd.DataFrame(
    valid_rois.groupby(['project_code', 'transgenic_line'])['id'].nunique()
).rename(columns = {'id': 'number of ROIs'})

In [14]:
cell_specimen_count = pd.DataFrame(
    valid_rois.groupby(['project_code', 'transgenic_line'])['cell_specimen_id'].nunique()
).rename(columns = {'cell_specimen_id': 'number of cells'})

## build a combined table

In [15]:
combined_table = mouse_count.merge(
    session_count,
    left_on = ['project_code','transgenic_line'],
    right_on = ['project_code','transgenic_line'],
).merge(
    experiment_count,
    left_on = ['project_code','transgenic_line'],
    right_on = ['project_code','transgenic_line'],
).merge(
    roi_count,
    left_on = ['project_code','transgenic_line'],
    right_on = ['project_code','transgenic_line'],
).merge(
    cell_specimen_count,
    left_on = ['project_code','transgenic_line'],
    right_on = ['project_code','transgenic_line'],
)

## get the totals of each column

In [16]:
totals = combined_table.sum()
totals = pd.DataFrame(
    {key:value for key, value in zip(totals.index, totals.values)},
    index = ['Total']
)
totals.index.name = 'project_code'
totals['transgenic_line'] = ''
totals = totals.reset_index().set_index(['project_code','transgenic_line'])

## display the final table

In [23]:
final_table = pd.concat((combined_table, totals))
final_table

number of mice  \
project_code             transgenic_line                          
VisualBehavior           Slc17a7-IRES2-Cre;Ai93              17   
                         Slc17a7-IRES2-Cre;Ai94               7   
                         Sst-IRES-Cre;Ai148                   4   
                         Vip-IRES-Cre;Ai148                   7   
VisualBehaviorMultiscope Slc17a7-IRES2-Cre;Ai93               6   
                         Sst-IRES-Cre;Ai148                   4   
                         Vip-IRES-Cre;Ai148                   9   
VisualBehaviorTask1B     Slc17a7-IRES2-Cre;Ai93              14   
                         Slc17a7-IRES2-Cre;Ai94               1   
                         Sst-IRES-Cre;Ai148                   7   
                         Vip-IRES-Cre;Ai148                   6   
Total                                                        82   

                                                 number of sessions  \
project_code             transgenic_line                              
VisualBehavior           Slc17a7-IRES2-Cre;Ai93                 103   
                         Slc17a7-IRES2-Cre;Ai94                  50   
                         Sst-IRES-Cre;Ai148                      26   
                         Vip-IRES-Cre;Ai148                      46   
VisualBehaviorMultiscope Slc17a7-IRES2-Cre;Ai93                  43   
                         Sst-IRES-Cre;Ai148                      30   
                         Vip-IRES-Cre;Ai148                      60   
VisualBehaviorTask1B     Slc17a7-IRES2-Cre;Ai93                  95   
                         Slc17a7-IRES2-Cre;Ai94                   6   
                         Sst-IRES-Cre;Ai148                      50   
                         Vip-IRES-Cre;Ai148                      42   
Total                                                           551   

                                                 number of experiments  \
project_code             transgenic_line                                 
VisualBehavior           Slc17a7-IRES2-Cre;Ai93                    103   
                         Slc17a7-IRES2-Cre;Ai94                     50   
                         Sst-IRES-Cre;Ai148                         26   
                         Vip-IRES-Cre;Ai148                         46   
VisualBehaviorMultiscope Slc17a7-IRES2-Cre;Ai93                    265   
                         Sst-IRES-Cre;Ai148                        149   
                         Vip-IRES-Cre;Ai148                        333   
VisualBehaviorTask1B     Slc17a7-IRES2-Cre;Ai93                     95   
                         Slc17a7-IRES2-Cre;Ai94                      6   
                         Sst-IRES-Cre;Ai148                         50   
                         Vip-IRES-Cre;Ai148                         42   
Total                                                             1165   

                                                 number of ROIs  \
project_code             transgenic_line                          
VisualBehavior           Slc17a7-IRES2-Cre;Ai93           15588   
                         Slc17a7-IRES2-Cre;Ai94           24340   
                         Sst-IRES-Cre;Ai148                 327   
                         Vip-IRES-Cre;Ai148                1176   
VisualBehaviorMultiscope Slc17a7-IRES2-Cre;Ai93           26813   
                         Sst-IRES-Cre;Ai148                1788   
                         Vip-IRES-Cre;Ai148                5353   
VisualBehaviorTask1B     Slc17a7-IRES2-Cre;Ai93           13938   
                         Slc17a7-IRES2-Cre;Ai94            1593   
                         Sst-IRES-Cre;Ai148                 577   
                         Vip-IRES-Cre;Ai148                 716   
Total                                                     92209   

                                                 number of cells  
project_code             transgenic_line                          
Visua